In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
# predict the state of a loan (Lending Club)
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

/home/glenn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
y2015.head(5)

id   member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68009401  72868139.0    16000.0      16000.0          16000.0   60 months   
1  68354783  73244544.0     9600.0       9600.0           9600.0   36 months   
2  68466916  73356753.0    25000.0      25000.0          25000.0   36 months   
3  68466961  73356799.0    28000.0      28000.0          28000.0   36 months   
4  68495092  73384866.0     8650.0       8650.0           8650.0   36 months   

  int_rate  installment grade sub_grade            ...              \
0   14.85%       379.39     C        C5            ...               
1    7.49%       298.58     A        A4            ...               
2    7.49%       777.55     A        A4            ...               
3    6.49%       858.05     A        A2            ...               
4   19.89%       320.99     E        E3            ...               

  num_tl_90g_dpd_24m num_tl_op_past_12m pct_tl_nvr_dlq  percent_bc_gt_75  \
0                0.0                2.0           78.9               0.0   
1                0.0                2.0          100.0              66.7   
2                0.0                0.0          100.0              20.0   
3                0.0                0.0           91.7              22.2   
4                0.0               12.0          100.0              50.0   

  pub_rec_bankruptcies tax_liens tot_hi_cred_lim total_bal_ex_mort  \
0                  0.0       2.0        298100.0           31329.0   
1                  0.0       0.0         88635.0           55387.0   
2                  0.0       0.0        373572.0           68056.0   
3                  0.0       0.0        304003.0           74920.0   
4                  1.0       0.0         38998.0           18926.0   

  total_bc_limit total_il_high_credit_limit  
0       281300.0                    13400.0  
1        12500.0                    75635.0  
2        38400.0                    82117.0  
3        41500.0                    42503.0  
4         2750.0                    18248.0  

[5 rows x 111 columns]

In [34]:
# check datatypes for distinct unique values
dtype_dic = {}
categorical = y2015.select_dtypes(include=["object"])
for i in categorical:
    dtype_dic[i] = categorical[i].nunique()
print(dtype_dic)

{'id': 421097, 'term': 2, 'int_rate': 110, 'grade': 7, 'sub_grade': 35, 'emp_title': 120812, 'emp_length': 11, 'home_ownership': 4, 'verification_status': 3, 'issue_d': 12, 'loan_status': 7, 'pymnt_plan': 1, 'url': 421095, 'desc': 34, 'purpose': 14, 'title': 27, 'zip_code': 914, 'addr_state': 49, 'earliest_cr_line': 668, 'revol_util': 1211, 'initial_list_status': 2, 'last_pymnt_d': 25, 'next_pymnt_d': 4, 'last_credit_pull_d': 26, 'application_type': 2, 'verification_status_joint': 3}


In [35]:
# convert ID and Interest Rate to numeric
y2015['id'] = pd.to_numeric(y2015['id'],errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'),errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [36]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X = y2015.drop("loan_status",1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)
Y = y2015["loan_status"]
#X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=0.7,test_size=0.3,random_state=50)
#display(X)

In [38]:
X.shape

(421095, 201)

In [39]:
scaler.fit(X)
X = scaler.transform(X)
pca = PCA(0.90)
pca.fit(X)
pca.n_components_

118

In [40]:
X = pca.transform(X)

In [41]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()

print("Cross_val_score:",cross_val_score(rfc,X,Y,cv=10))

KeyboardInterrupt: 

Reduced features from 201 to 118 (90% variance retained)